# Loading Multiple CSV format files into SQL Server

In [1]:
!pip install pypyodbc

In [157]:
#Importing required packages to process

import os
import pyodbc as odbc
import pandas as pd

# Renaming the inconsistent column name by removing Leading Space by iteration

In [184]:
# Data Cleansing

data_file_folder = os.path.join(os.getcwd(), 'Bhavcopy')
data_files = os.listdir(path = data_file_folder)
for data_file in data_files:
    # Remove spaces from column headers
    print(os.path.join(data_file_folder, data_file))
    df = pd.read_csv(os.path.join(data_file_folder, data_file))
    print(df.columns)
    #Removing the space of column names in csv files.
    df.rename(columns={' SERIES': 'SERIES', ' DATE1': 'DATE1', ' PREV_CLOSE': 'PREV_CLOSE', ' OPEN_PRICE': 'OPEN_PRICE',
                       ' HIGH_PRICE': 'HIGH_PRICE',' LOW_PRICE': 'LOW_PRICE', ' LAST_PRICE': 'LAST_PRICE',
                       ' CLOSE_PRICE': 'CLOSE_PRICE', ' AVG_PRICE': 'AVG_PRICE', ' TTL_TRD_QNTY': 'TTL_TRD_QNTY',
                       ' TURNOVER_LACS': 'TURNOVER_LACS', ' NO_OF_TRADES': 'NO_OF_TRADES', ' DELIV_QTY': 'DELIV_QTY',
                       ' DELIV_PER': 'DELIV_PER'
                      }, 
              inplace=True)

    
    #Handling incosistent data in column names in csv files.
    df = df.replace(' -','')
    
    #Applying the changes on base location
    df.to_csv(os.path.join(data_file_folder, data_file),index = False)

C:\Users\Admin\Automation_Python\Bhavcopy\sec_bhavdata_full_02042024.csv
Index(['SYMBOL', 'SERIES', 'DATE1', 'PREV_CLOSE', 'OPEN_PRICE', 'HIGH_PRICE',
       'LOW_PRICE', 'LAST_PRICE', 'CLOSE_PRICE', 'AVG_PRICE', 'TTL_TRD_QNTY',
       'TURNOVER_LACS', 'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER'],
      dtype='object')
C:\Users\Admin\Automation_Python\Bhavcopy\sec_bhavdata_full_03042024.csv
Index(['SYMBOL', 'SERIES', 'DATE1', 'PREV_CLOSE', 'OPEN_PRICE', 'HIGH_PRICE',
       'LOW_PRICE', 'LAST_PRICE', 'CLOSE_PRICE', 'AVG_PRICE', 'TTL_TRD_QNTY',
       'TURNOVER_LACS', 'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER'],
      dtype='object')
C:\Users\Admin\Automation_Python\Bhavcopy\sec_bhavdata_full_04042024.csv
Index(['SYMBOL', 'SERIES', 'DATE1', 'PREV_CLOSE', 'OPEN_PRICE', 'HIGH_PRICE',
       'LOW_PRICE', 'LAST_PRICE', 'CLOSE_PRICE', 'AVG_PRICE', 'TTL_TRD_QNTY',
       'TURNOVER_LACS', 'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER'],
      dtype='object')
C:\Users\Admin\Automation_Python\Bhavcopy\sec_

# Function to BULK INSERT statement to import data into SQL Server

In [185]:
def csv_bulk_insert(data_file, target_sql_table):
    sql = f"""
            BULK INSERT {target_sql_table}
            from '{data_file}'
            WITH (
                FORMAT = 'CSV',
                FIRSTROW = 2,
                FIELDTERMINATOR  = ',',
                ROWTERMINATOR = '0x0a'
            )
            """.strip()
    return sql
            

In [186]:
#Grouping the connection parameters

DATABASE_NAME = 'NSEBhavcopy'
target_sql_table = 'nse_dailydata_bhav_copy'
SERVER_NAME = 'DESKTOP-EQ55Q8H'


In [187]:
#SQL Server Connection Establishment
conn = odbc.connect(f"""
    Driver={{SQL Server}};
    Server={SERVER_NAME};
    Database={DATABASE_NAME};
    # uid=<user id>;
    # pwd=<password>;
    """.strip())

print(conn)

In [188]:
os.getcwd()

'C:\\Users\\Admin\\Automation_Python'

In [189]:
data_file_folder = os.path.join(os.getcwd(), 'Bhavcopy')
print(data_file_folder)


C:\Users\Admin\Automation_Python\Bhavcopy


In [190]:
data_files = os.listdir(path = data_file_folder)
data_files

['sec_bhavdata_full_02042024.csv',
 'sec_bhavdata_full_03042024.csv',
 'sec_bhavdata_full_04042024.csv',
 'sec_bhavdata_full_05042024.csv',
 'sec_bhavdata_full_08042024.csv',
 'sec_bhavdata_full_09042024.csv',
 'sec_bhavdata_full_10042024.csv',
 'sec_bhavdata_full_11042024.csv']

# Iterating through the base location to upload data in SQL server

In [192]:
cursor = conn.cursor()

try:
    # here we can use with statement to automatically close connection once the operation is complete
    with cursor:
        for data_file in data_files:
            if data_file.endswith('.csv'):
                cursor.execute(csv_bulk_insert(os.path.join(data_file_folder, data_file), target_sql_table))
                #print(os.path.join(data_file_folder, data_file), target_sql_table + ' inserted')
        cursor.commit()
except Exception as e:
    print(e)
    conn.rollback()
    print('Transaction rollback')